# Iterators
- An iterator is an object that allows you to traverse through all the elements of a collection (like lists, tuples, etc.) one at a time without needing to know the underlying structure.

### Iterator Protocol
- (__iter__()) returns the iterator object itself.
- (__next__()) returns the next value from the container. If no more elements, it raises StopIteration

In [2]:
# Iterable vs Iterator
# 1. Iterable: An object capable of returning its members one at a time (e.g., list, tuple, string). Must implement __iter__() method.
# 2. Iterator: An object with a __next__() method that returns the next element and __iter__() that returns itself.

lst = [1, 2, 3]
it = iter(lst)         # Converts list into iterator

print(next(it))        # Output: 1
print(next(it))        # Output: 2
print(next(it))        # Output: 3
print(next(it))        # Raises StopIteration

1
2
3


StopIteration: 

In [4]:
# Creating a Custom Iterator Class
class CountUpTo:
    def __init__(self, max):
        self.max = max
        self.num = 1

    def __iter__(self):
        return self  # returns iterator object (self)

    def __next__(self):
        if self.num <= self.max:
            val = self.num
            self.num += 1
            return val
        else:
            raise StopIteration

counter = CountUpTo(5)
for number in counter:
    print(number)

1
2
3
4
5


In [5]:
# Iterating Manually Using next()
it = iter([10, 20, 30])
print(next(it))   # 10
print(next(it))   # 20
print(next(it))   # 30
print(next(it))   # Raises StopIteration

10
20
30


StopIteration: 

In [6]:
# Behind the Scenes of for Loop
lst = [1, 2, 3]
it = iter(lst)
while True:
    try:
        item = next(it)
        print(item)
    except StopIteration:
        break

1
2
3


# Generator 
- A generator is a special type of iterator that allows you to iterate through data lazily (one item at a time, only when needed), using the yield keyword instead of return.
- Generators are memory-efficient and pause & resume function execution.

In [7]:
def simple_gen():
    yield 1
    yield 2
    yield 3

for val in simple_gen():
    print(val)

1
2
3


In [11]:
# How Generator Works Internally
# Each time you call next() on a generator, the function runs until it hits the next yield, then pauses and remembers its state.
def count_up_to(n):
    count = 1
    while count <= n:
        yield count
        count += 1

counter = count_up_to(3)
print(next(counter))  # 1
print(next(counter))  # 2
print(next(counter))  # 3
# print(next(counter))  # StopIteration ( It thows an error )

1
2
3


In [15]:
# Generator Expressions
# Generator expression
gen = (x*x for x in range(5))

print(next(gen))  # 0
print(next(gen))  # 1
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))  # here it ended so thats why it showing me error 

0
1
4
9
16


StopIteration: 

In [23]:
# Comperasition of List Comprehension and generator expression 
# List comprehension (loads entire list in memory)
lst = [x*x for x in range(10)]
print(lst)

# Generator (one at a time)
gen = (x*x for x in range(1000000))   # here we can use the amout of data is needed only.
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
0
1
4
9


# Partial Functions (from functools.partial)
- A partial function is a new function with some arguments of the original function fixed.
It’s created using the functools.partial() method.

In [4]:
from functools import partial

def power(base, exponent):
    return base ** exponent

# Create a new function where exponent is fixed to 2
square = partial(power, exponent=2)
print(square(5))  # Output: 25


int_base2 = partial(int, base=2)
print(int_base2('1010'))

25
10


# Curring
- Currying is the process of converting a function that takes multiple arguments into a series of functions that take one argument at a time.
- Python doesn’t have built-in currying like functional languages, but you can implement it manually or with nested functions.

In [7]:
# Manual Currying Example
def add(x):
    def inner(y):
        return x + y
    return inner
add5 = add(5)
print(add5(3))  # Output: 8

# Curring with Lambda Function
add = lambda x: (lambda y: x + y)
print(add(2)(3))  # Output: 5


8
5


# MEMORY MANAGEMENT

### Reference Counting
- Python keeps track of the number of references pointing to an object in memory.
- When the reference count drops to 0, the object is immediately deallocated.

In [11]:
import sys

a = []       # a points to list
b = a        # b also points to the same list
print(sys.getrefcount(a))  # 3 (a, b, getrefcount arg) {sys.getrefcount(obj) returns the reference count (add +1 because it's passed as an argument).}

del a
print(sys.getrefcount(b))  # 2

3
2


###  Garbage Collection
-  Why is GC needed if we have reference counting?
- Because cyclic references can't be handled by reference counting alone.

In [14]:
# Example of an cyclic counting
class Node:
    def __init__(self):
        self.ref = self

n = Node()
print(n)
del n
# print(n) it will throw an error as i have deleted the variable

### Python’s gc (Garbage Collector) handles such cycles.

- | Tool/Function                  | Purpose                                 |
- | --------------------------- | ---------------------------------- |
- | `sys.getsizeof(obj)`           | Size (in bytes) of object               |
- | `sys.getrefcount(obj)`         | Reference count of object               |
- | `gc.collect()`                 | Manually trigger garbage collection (Forces garbage collection immediately)    |
- | `gc.get_count()`               | Return current counts of GC generations (Returns count of tracked objects for 3 generations) |
- | `gc.set_threshold()` gc.set_threshold(threshold0, threshold1, threshold2)    | Control collection frequency   Set when collection happens for each generation         |
- | `gc.get_objects()`             | Inspect tracked objects (Returns a list of all objects tracked by the GC)                 |
- | `gc.disable()` / `gc.enable()` | Turn GC off/on                          |


In [23]:
import gc

class Node:
    def __init__(self, name):
        self.name = name
        self.ref = None

    def __del__(self):
        print(f"Deleted: {self.name}")

# Enable debug to watch GC
gc.set_debug(gc.DEBUG_UNCOLLECTABLE)

# Create two nodes with circular reference
n1 = Node("Node1")
n2 = Node("Node2")
n1.ref = n2
n2.ref = n1

# Remove external references
n1 = None
n2 = None

# Force garbage collection
unreachable = gc.collect()

print(f"Unreachable objects: {unreachable}")
print(f"Garbage objects: {gc.garbage}")


Deleted: Node1
Deleted: Node2
Unreachable objects: 8
Garbage objects: []


In [25]:
import objgraph

class A:
    def __init__(self):
        self.other = None

a = A()
b = A()
a.other = b
b.other = a

objgraph.show_backrefs([a], filename='circular_ref.png')

Graph written to C:\Users\Dev\AppData\Local\Temp\objgraph-qacnd1pn.dot (24 nodes)
Image renderer (dot) not found, not doing anything else


### sys.getsizeof() — Check Size of an Object

In [31]:
import sys

a = "Hello, this is Devashish Sharma"
print(sys.getsizeof(a))  

72
